# Watson-Natural Language Understanding

### Ejemplo de implementacion de aplicacion Watson-Natural Language Understanding via api

#### En este ejemplo se tratara de analizar los sentimientos contenidos dentro de twitts con respecto a xxx .
#### Tambien se analizaran las emociones contenidas dentro de twitts con respecto a xxx .
<br>

--------------------------------------------------------------------------------------------------------------

#### Que es lo que analizamos ? [Segun Wikipedia](https://es.wikipedia.org/wiki/An%C3%A1lisis_de_sentimiento)
##### Basicamente, determinar la actitud de un interlocutor o usuario con respecto a algún tema o la polaridad (negativa o positiva) contextual general de un documento.
<br>

--------------------------------------------------------------------------------------------------------------

#### Librerias especiales implementadas
<ul>
<li><strong>tweepy</strong>, para captar twitts y facilitar la autenticacion via api</li>
<li><strong>ibm_watson</strong>, para analisis de sentimientos/ emociones</li>
<br>

In [215]:
import pandas as pd
import tweepy
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, CategoriesOptions

<br>

## Primera etapa: Obtener tweets
<br>

####  Authentication en la api de twitter.

In [216]:
# Authentication
consumerKey = '-'
consumerSecret = '-'
accessToken = '-'
accessTokenSecret = '-'

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

<br>

#### Un poco de referencia del metodo que se utilizara para buscar twitts de los temas.
#####  [tweepy.API().search()](https://docs.tweepy.org/en/latest/api.html?highlight=tweepy.API#API.search)

##### (Metodo premium)
##### [API.search_30_day()](https://docs.tweepy.org/en/latest/api.html?highlight=API.search_30_day#API.search_30_day)
<br>

-------------------------------------------

#### Se inicia una lista vacia que se llenara con los twitts que contengan las palabras que le indiquemos al buscador.

In [217]:
list_tweets = []

### Primer tema de busqueda
search_tweets = api.search('covid', lang="es", count=100, tweet_mode='extended')

for tweet in search_tweets:
    if 'retweeted_status' in tweet._json:
        list_tweets.append(tweet._json['retweeted_status']['full_text'].upper())
        # print(tweet._json['retweeted_status']['full_text'])
    else:
        list_tweets.append(tweet.full_text.upper())
        # print(tweet.full_text)

### Quitamos los tweets spam.
list_tweets = list(set(list_tweets))

<br>

--------------------------------------------------
#### Seccion para visualizar que obtuvimos.

<br>

In [218]:
print(len(list_tweets))

94


In [219]:
for i in list_tweets:
    print(i, '\n')
    break

@BELTRANDELRIO SOBRE LA INFORMACIÓN QUE DIO @RODPAC DE QUE LAS PERSONAS QUE VIAJEN A MÉXICO DESDE UK DEBERÁN A SU REGRESO HACER UNA CUARENTENA EN UN HOTEL, TE COMENTO QUE MÉXICO NO ESTÁ INCLUIDO EN LA LISTA QUE PUBLICÓ EL GOBIERNO DE UK, SALUDOS.

HTTPS://T.CO/TZ9OVTSRGX 



<br>

## Segunda etapa: Analizis con Watson

<br>

###  Autenticacion en la api de IBM-Watson.
#### [documentacion](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#authentication)

In [220]:
authenticator = IAMAuthenticator('-')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

### Configuracion de endpoint
#### [documentacion](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#service-endpoint)

In [221]:
service = '-'
natural_language_understanding.set_service_url(service)

<br>

#### Metodo para analizar el sentimiento de los textos.
https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#sentiment

#### Metodo para clasificar los textos.
https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#categories

### Doc
[natural_language_understanding()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html)

[analyse()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=analyze#ibm_watson.natural_language_understanding_v1.NaturalLanguageUnderstandingV1.analyze)

[Features()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=features#ibm_watson.natural_language_understanding_v1.Features)

[SentimentOptions()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=sentimentoptions#ibm_watson.natural_language_understanding_v1.SentimentOptions)

[CategoriesOptions()](http://watson-developer-cloud.github.io/python-sdk/v3.0.2/apis/ibm_watson.natural_language_understanding_v1.html?highlight=ategoriesoptions#ibm_watson.natural_language_understanding_v1.CategoriesOptions)

<br>

#### Un ejemplo de como funciona este metodo.

In [223]:
response = natural_language_understanding.analyze(
                                                language='es',
                                                text='las naranjas son amargas y feas',
                                                features=Features(
                                                                sentiment=SentimentOptions(targets=['manzanas', 'naranjas'])
                                                                # ,categories=CategoriesOptions(limit=3)
                                                    )
                                                )


print(json.dumps(response.get_result(), indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 31,
    "features": 1
  },
  "sentiment": {
    "targets": [
      {
        "text": "naranjas",
        "score": -0.888776,
        "label": "negative"
      }
    ],
    "document": {
      "score": -0.888776,
      "label": "negative"
    }
  },
  "language": "es"
}


<br>

#### Para entender cada campo del response que obtenemos -> [Doc Response](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#analyze)

<br>

-----------------------
<br>

####  Scoring y clasificasión de tweets

<br>

In [224]:
data = []

for tweet in list_tweets:
    response = natural_language_understanding.analyze(
                                                language='es',
                                                text=tweet,
                                                features=Features(
                                                                sentiment=SentimentOptions(targets=[tweet])
                                                    )
                                                )


    json_df = response.get_result()
    json_parse = json_df["sentiment"]["targets"][0]
    data.append(json_parse)

In [251]:
df = pd.DataFrame(data)

In [252]:
df.head()

,text,score,label,mixed
0,@BELTRANDELRIO SOBRE LA INFORMACIÓN QUE DIO @R...,0.000000,neutral,NaN
1,EL HOSPITAL REGIONAL MARIA AUXILIADORA INFORM...,-0.566191,negative,NaN
2,"POR REBROTE DEL COVID, PIB CAERÁ 1% A MARZO\n ...",0.000000,neutral,NaN
3,¿TIENES O TUVISTE COVID - 19? UPAEP TE INVITA ...,0.000000,neutral,NaN
4,#CHIMALHUACÁN SIGUE EN PIE DE LUCHA ANTE EL CO...,0.000000,neutral,NaN


In [253]:
index = 13
print('TEXTO:', df['text'].iloc[index], '\n')
print('PUNTUACIÓN:', df['score'].iloc[index], '\n')
print('ETIQUETA:', df['label'].iloc[index], '\n')

TEXTO: LO VEN??
O NO LO QUIEREN VER??
YA DIÓ NEGATIVO EN LA PRUEBA DE COVID..

LOS FANÁTICOS Y LOS BOTS PAGADOS DIRÁN QUE ES UN MILAGRO..
DIRÁN QUE SU "FUERZA MORAL" LO PROTEGE..
DIRÁN QUE ES EL MESÍAS..

NOSOTRAS, SABEMOS LA VERDAD..

TODO FUE UNA MENTIRA!!.

#VIEJOMENTIROSO 

PUNTUACIÓN: 0.27392 

ETIQUETA: positive 

